In [24]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex
import libsbml
import pandas as pd
import copy


In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


In [3]:
#Solver
model.solver = 'cplex'

In [4]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)

mm.read_medium_csv('../misc/photo_medium.csv', model)

#Test natin kung magwowork sa est. infinite
#Yes setting bounds to 1000 works



for imports in model.exchanges:
    if imports.upper_bound == 1000:
        model.reactions.get_by_id(imports.id).upper_bound = inf
    elif imports.lower_bound == -1000:
        print(imports.id, imports.lower_bound)
        model.reactions.get_by_id(imports.id).lower_bound = -inf
        print(imports.id, imports.lower_bound)
#Print out active import fluxes

model_media = model.medium
for items in model.exchanges:
    print(items.id, items.bounds)

for imports in model.exchanges:
    if imports.upper_bound == 1000:
        model.reactions.get_by_id(imports.id).upper_bound = inf
    elif imports.lower_bound == -1000:
        print(imports.id, imports.lower_bound)
        model.reactions.get_by_id(imports.id).lower_bound = -inf
        print(imports.id, imports.lower_bound)


EX_co2_LPAREN_e_RPAREN__M (-1000000, 0.0)
EX_h2o_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_h_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_no3_LPAREN_e_RPAREN__M (-1000000, 0.0)
EX_o2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_pi_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_sucr_LPAREN_e_RPAREN__M (0, 0.0)
EX_fru_DASH_B_LPAREN_e_RPAREN__M (0, 0.0)
EX_glc_DASH_A_LPAREN_e_RPAREN__M (0, 0.0)
EX_so4_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_hco3_LPAREN_e_RPAREN__M (0, 0.0)
EX_so3_LPAREN_e_RPAREN__M (-1000000, 0.0)
EX_h2s_LPAREN_e_RPAREN__M (0, 0.0)
EX_nh4_LPAREN_e_RPAREN__M (0, 0.0)
EX_asn_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_gln_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_etoh_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_ac_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_lac_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_tsul_LPAREN_e_RPAREN__M (0, 0.0)
EX_fe2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_fe3_LPAREN_e_RPAREN__M (0, 0.0)
EX_mg2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_ala_DASH_L_LPARE

In [5]:
#Modifications to medium

model_media['EX_photonVis_LPAREN_e_RPAREN__M'] = 1000 #PPFD used in Ermakova et al for high light
model_media['EX_photonVis_LPAREN_e_RPAREN__BS'] = 1000 #PPFD used in Ermakova et al for high light 

#Set CO2 bounds to ~29 m-2 s-1
model_media['EX_co2_LPAREN_e_RPAREN__M'] = 30
model_media['EX_co2_LPAREN_e_RPAREN__BS'] = 30
# model_media['EX_o2_LPAREN_e_RPAREN__M'] = inf
# model_media['EX_o2_LPAREN_e_RPAREN__BS'] = inf

model.medium = model_media

In [22]:
for item in model.exchanges:
    print(item.id, item.bounds)

EX_co2_LPAREN_e_RPAREN__M (-30, 0.0)
EX_h2o_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_h_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_no3_LPAREN_e_RPAREN__M (-1000000, 0.0)
EX_o2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_pi_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_sucr_LPAREN_e_RPAREN__M (0, 0.0)
EX_fru_DASH_B_LPAREN_e_RPAREN__M (0, 0.0)
EX_glc_DASH_A_LPAREN_e_RPAREN__M (0, 0.0)
EX_so4_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_hco3_LPAREN_e_RPAREN__M (0, 0.0)
EX_so3_LPAREN_e_RPAREN__M (-1000000, 0.0)
EX_h2s_LPAREN_e_RPAREN__M (0, 0.0)
EX_nh4_LPAREN_e_RPAREN__M (0, 0.0)
EX_asn_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_gln_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_etoh_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_ac_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_lac_DASH_L_LPAREN_e_RPAREN__M (0, 1000000.0)
EX_tsul_LPAREN_e_RPAREN__M (0, 0.0)
EX_fe2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_fe3_LPAREN_e_RPAREN__M (0, 0.0)
EX_mg2_LPAREN_e_RPAREN__M (-1000000, 1000000.0)
EX_ala_DASH_L_LPAREN_e_R

In [23]:
#Add constraints to model

#BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
BS_photon_import = model.reactions.get_by_id("EX_photonVis_LPAREN_e_RPAREN__BS")
M_photon_import = model.reactions.get_by_id("EX_photonVis_LPAREN_e_RPAREN__M")

photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
                                       - BS_photon_import.flux_expression,
                                       lb = 0, ub = 1000)
model.add_cons_vars(photon_flux)


#Note: CO2 uptake is not restricted in BS cells but will be constrained to be  equal/less than M cell
BS_co2_import = model.reactions.get_by_id("EX_co2_LPAREN_e_RPAREN__BS") 
M_co2_import = model.reactions.get_by_id("EX_co2_LPAREN_e_RPAREN__M")
co2_flux_cons = model.problem.Constraint(M_co2_import.flux_expression - BS_co2_import.flux_expression, lb = 0, ub = inf)
model.add_cons_vars(co2_flux_cons)

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
co2_import_cons = model.problem.Constraint(M_co2_import.flux_expression + BS_co2_import.flux_expression, lb = -30.2, ub = -27.8)
model.add_cons_vars(co2_import_cons)
#Oks na
#CO2 intake needs to be mostly in the M cells. Not sure of the exact value: 
#try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
#I'll assume that the ratio is at 9.5:0.5 in the meantime, meaning that there is minimal gas exchange into the BS cell

co2_intake_ratio = mm.set_fix_flux_ratio({M_co2_import.id:4, BS_co2_import.id:1},model)
#Automatic na palang nalalagay siya as constraint

#First is the BS and M Cell constraints

In [8]:
model.remove_cons_vars(co2_intake_ratio)

In [9]:
# #Let's try constraining CO2 flux
# BS_co2_import.bounds = (0, 0)
# M_co2_import.bounds = (0, 0)

# BS_photon_import.bounds = (0, 0)
# M_photon_import.bounds = (0, 0)

# print('BS_photon', model.reactions.get_by_id('EX_photonVis_LPAREN_e_RPAREN__BS').bounds)
# print('M_photon', model.reactions.get_by_id('EX_photonVis_LPAREN_e_RPAREN__M').bounds)

In [10]:
#What if I restrict yung import ng O2?

M_o2_import = model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN__M')
BS_o2_import = model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN__BS')
#Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
o2_import_cons = model.problem.Constraint(M_o2_import.flux_expression + BS_o2_import.flux_expression, lb = -3.312, ub = 0)
model.add_cons_vars(o2_import_cons)


In [11]:
model.optimize()

,fluxes,reduced_costs
Coleoptile_Biomass_M,0.000000,-2.755272e+00
Straw_Biomass_M,0.000000,-2.664535e-15
ACCOAC_OSAc_M,0.000000,-2.775558e-17
ACCOAC_OSAr_M,0.000000,1.112999e-14
ACPC_OSs_M,0.000000,0.000000e+00
...,...,...
g3p_pd,0.000000,0.000000e+00
3pg_pd,279.184443,0.000000e+00
glyclt_pd,0.000000,-0.000000e+00
acon_DASH_C_pd,0.000000,0.000000e+00


In [12]:
sample_fluxes = cobra.flux_analysis.pfba(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_LPAREN_e_RPAREN__BS,30,1,99.34%
co2_e,EX_co2_LPAREN_e_RPAREN__M,0.2,1,0.66%
h_e,EX_h_LPAREN_e_RPAREN__M,2000,0,0.00%
no3_e,EX_no3_LPAREN_e_RPAREN__BS,121.1,0,0.00%
o2_e,EX_o2_LPAREN_e_RPAREN__M,3.312,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__BS,1000,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__M,1000,0,0.00%
pi_e,EX_pi_LPAREN_e_RPAREN__M,1.408,0,0.00%
so4_e,EX_so4_LPAREN_e_RPAREN__M,0.07435,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [13]:
for items in model.reactions:
    print(items.id, sample_fluxes[items.id])

Coleoptile_Biomass_M 0.0
Straw_Biomass_M 3.3944727954396168
ACCOAC_OSAc_M 0.0
ACCOAC_OSAr_M 0.3427127623672181
ACPC_OSs_M 5.956513060567659e-12
ACPC_OSm_M 0.0
FAC_Osc_M 0.0
FAC_Osr_M -0.009674247467002909
EX_co2_LPAREN_e_RPAREN__M -0.1999999999999993
EX_h2o_LPAREN_e_RPAREN__M 0.0
EX_h_LPAREN_e_RPAREN__M -2000.0
EX_no3_LPAREN_e_RPAREN__M 0.0
EX_o2_LPAREN_e_RPAREN__M -3.312
EX_pi_LPAREN_e_RPAREN__M 0.0
EX_sucr_LPAREN_e_RPAREN__M 0.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M 0.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M 0.0
EX_so4_LPAREN_e_RPAREN__M -0.07435020121002109
EX_hco3_LPAREN_e_RPAREN__M 0.0
EX_so3_LPAREN_e_RPAREN__M 0.0
EX_h2s_LPAREN_e_RPAREN__M 0.0
EX_nh4_LPAREN_e_RPAREN__M 0.0
EX_asn_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gln_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_etoh_LPAREN_e_RPAREN__M 0.0
EX_ac_LPAREN_e_RPAREN__M 0.0
EX_lac_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_tsul_LPAREN_e_RPAREN__M 0.0
EX_fe2_LPAREN_e_RPAREN__M 0.0
EX_fe3_LPAREN_e_RPAREN__M 0.0
EX_mg2_LPAREN_e_RPAREN__M 0.0
EX_ala_DASH_L_LPAREN_e_RPAREN__

UGALCGc_M 0.0
GLB1c_M 0.0
OACPSSTm_M 0.0
LACPLTm_M 0.0
MCOAMTc_M 0.0
MACPMMEc_M 0.0
3HGNORc_M 0.0
3HGMELc_M 0.0
GLUACPMEORc_M 0.0
GLUMEORc_M 0.0
HPIMMEORc_M 0.0
HPIMMELc_M 0.0
PIMACPMEc_M 0.0
PACPMEEc_M 0.0
AONSc_M 0.0
AMETAOTc_M 0.0
DTBTSc_M 0.0
BTNSc_M 0.0
PFRTc_M 0.0
CAAPc_M 0.0
PIOCMTc_M 0.0
FRCYOc_M 0.0
FRNCTIc_M 0.0
FRORc_M 0.0
FRPTc_M 0.0
FRPPTc_M 0.0
DEDOLDPSc_M 0.0
DEDOLDPSATc_M 0.0
DOLDPPc_M 0.0
DOLKc_M 0.0
GLCNACPTc_M 0.0
GLCNACTc_M 0.0
BDMTc_M 0.0
DOLPMTc_M 0.0
G13MTc_M 0.0
G16MT1c_M 0.0
G12MT1c_M 0.0
G12MT2c_M 0.0
DOLPMT1c_M 0.0
G12MT3c_M 0.0
G16MT2c_M 0.0
G12MT4c_M 0.0
UDPDOLPTc_M 0.0
DOLPGT1c_M 0.0
DOLPGT2c_M 0.0
DOLPGT3c_M 0.0
DOLASNTc_M 0.0
MG1Ac_M 0.0
MG1Bc_M 0.0
MG2Ac_M 0.0
MG2Bc_M 0.0
MG3Ac_M 0.0
MG3Bc_M 0.0
MM1c_M 0.0
MGAGT1c_M 0.0
MM2c_M 0.0
MGAGT2c_M 0.0
MGAGT3c_M 0.0
GPXTc_M 0.0
GPFTc_M 0.0
PIACGTc_M 0.0
ACGPIDc_M 0.0
GPIATc_M 0.0
GPIMTc_M 0.0
H6ET3c_M 0.0
H2MTc_M 0.0
H3MTc_M 0.0
H4ETc_M 0.0
H6MTc_M 0.0
A4GALTc_M 0.0
B3GALT3c_M 0.0
AGALSc_M 0.0
HXAMS1c_M 0.0
SIA

MTE160m_BS 0.0
MCOATAc_BS 0.0
MCOATAm_BS 0.0
ACOATAm_BS 0.0
MACPCLm_BS 0.0
3OAR40m_BS 0.0
3HAD40m_BS 0.0
EAR40ym_BS 0.0
3OAS60m_BS 0.0
3OAR60m_BS 0.0
3HAD60m_BS 0.0
EAR60ym_BS 0.0
3OAS80m_BS 0.0
3OAR80m_BS 0.0
3HAD80m_BS 0.0
EAR80ym_BS 0.0
3OAS100m_BS 0.0
3OAR100m_BS 0.0
3HAD100m_BS 0.0
EAR100ym_BS 0.0
3OAS120m_BS 0.0
3OAR120m_BS 0.0
3HAD120m_BS 0.0
EAR120ym_BS 0.0
3OAS140m_BS 0.0
3OAR140m_BS 0.0
3HAD140m_BS 0.0
EAR140ym_BS 0.0
3OAS160m_BS 0.0
3OAR160m_BS 0.0
3HAD160m_BS 0.0
EAR160ym_BS 0.0
3OAS180m_BS 0.0
3OAR180m_BS 0.0
3HAD180m_BS 0.0
EAR180ym_BS 0.0
FAS161ACP_Lm_BS 0.0
FAS181ACP_Lm_BS 0.0
FAS182ACP_Lm_BS 0.0
FAS183ACP_Lm_BS 0.0
G3PDc_BS 0.0
GLYCKr_BS 0.0
GLYCKs_BS 0.0
GLYCKm_BS 0.0
G3PDNc_BS -0.12192351202085389
G3PATr_BS 0.12192351202085386
G3PATs_BS 0.0
G3PATm_BS 0.0
AGPATr_BS 0.12192351202085386
AGPATs_BS 0.0
AGPATm_BS 0.0
PAPAr_BS 0.11376466651000107
PAPAs_BS 0.0
PAPAm_BS 0.0
DAGKm_BS 0.0
DAGACTr_BS 0.10211406813247086
TAGLc_BS 0.0
DAGLc_BS 0.0
MAGLc_BS 0.0
CDPDAGSr_BS 0.007504

In [18]:
#Exchange reactions
for exs in model.exchanges:
    print(exs.id, sample_fluxes[exs.id])

EX_co2_LPAREN_e_RPAREN__M -0.1999999999999993
EX_h2o_LPAREN_e_RPAREN__M 0.0
EX_h_LPAREN_e_RPAREN__M -2000.0
EX_no3_LPAREN_e_RPAREN__M 0.0
EX_o2_LPAREN_e_RPAREN__M -3.312
EX_pi_LPAREN_e_RPAREN__M 0.0
EX_sucr_LPAREN_e_RPAREN__M 0.0
EX_fru_DASH_B_LPAREN_e_RPAREN__M 0.0
EX_glc_DASH_A_LPAREN_e_RPAREN__M 0.0
EX_so4_LPAREN_e_RPAREN__M -0.07435020121002109
EX_hco3_LPAREN_e_RPAREN__M 0.0
EX_so3_LPAREN_e_RPAREN__M 0.0
EX_h2s_LPAREN_e_RPAREN__M 0.0
EX_nh4_LPAREN_e_RPAREN__M 0.0
EX_asn_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gln_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_etoh_LPAREN_e_RPAREN__M 0.0
EX_ac_LPAREN_e_RPAREN__M 0.0
EX_lac_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_tsul_LPAREN_e_RPAREN__M 0.0
EX_fe2_LPAREN_e_RPAREN__M 0.0
EX_fe3_LPAREN_e_RPAREN__M 0.0
EX_mg2_LPAREN_e_RPAREN__M 0.0
EX_ala_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_arg_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_asp_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_cys_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_glu_DASH_L_LPAREN_e_RPAREN__M 0.0
EX_gly_LPAREN_e_RPAREN__M 0.0
EX_his_DASH_L_LPAREN_

In [19]:
model.reactions.CC3H1c_M

Reaction identifier,CC3H1c_M
Name,p-coumarate:caffeate 3-hydroxylase
Memory address,0x07fdcfd13fd30
Stoichiometry,coum_c0 + nadph_c0 + o2_c0 --> caff_c0 + h2o_c0 + nadp_c0 4-Coumarate + Nicotinamide adenine dinucleotide phosphate - reduced + O2 --> Caffeate + H2O + Nicotinamide adenine dinucleotide phosphate
GPR,LOC_Os05g41440
Lower bound,0.0
Upper bound,1000.0


In [20]:
#Show all pd reactions
pd_rxns = [rxn for rxn in model.reactions if "_pd" in rxn.id]

for items in pd_rxns:
    print(items.id, sample_fluxes[items.id])

ala_DASH_L_pd 0.0
arg_DASH_L_pd 0.0
asp_DASH_L_pd 0.0
cys_DASH_L_pd 0.0
glu_DASH_L_pd 0.0
gly_pd 0.0
his_DASH_L_pd 0.0
ile_DASH_L_pd 0.0
leu_DASH_L_pd 0.0
lys_DASH_L_pd 0.0
met_DASH_L_pd 0.0
phe_DASH_L_pd 0.0
pro_DASH_L_pd 0.0
ser_DASH_L_pd 0.0
thr_DASH_L_pd 0.0
trp_DASH_L_pd 0.0
tyr_DASH_L_pd 0.0
val_DASH_L_pd 0.0
glc_DASH_A_pd 0.0
sucr_pd 0.0
fru_DASH_B_pd 0.0
udparab_pd 0.0
udpxyl_pd 0.0
udpgal_pd 0.0
udpg_pd 0.0
udpglcur_pd 0.0
co2_pd 0.0
h2o_pd 0.0
o2_pd 0.0
pep_pd 0.0
pyr_pd 0.0
mal_DASH_L_pd 0.0
accoa_pd 0.0
glu5p_pd 0.0
akg_pd 0.0
icit_pd 0.0
gal_pd 0.0
g1p_pd 0.0
tre_pd 0.0
glc_DASH_B_pd 0.0
tre6p_pd 0.0
g6p_DASH_A_pd 0.0
b_DASH_glucan_pd 0.0
mnt_pd 0.0
melib_pd 0.0
stc_pd 0.0
raffin_pd 0.0
1Dgali_pd 0.0
inost_pd 0.0
cit_pd 0.0
oxa_pd 0.0
2pg_pd 0.0
dhap_pd 0.0
g3p_pd 0.0
3pg_pd 0.0
glyclt_pd 0.0
acon_DASH_C_pd 0.0
succ_pd 0.0


In [21]:
#Let's try to do initial FBA
model.metabolites.gly_c0.summary()


Percent,Flux,Reaction,Definition
85.19%,19.87,GLYtm_M,gly_m0 + h_m0 <=> gly_c0 + h_c0
14.81%,3.456,THRAc_M,thr_DASH_L_c0 --> acald_c0 + gly_c0
Percent,Flux,Reaction,Definition
46.05%,-10.74,GLYts_M,gly_c0 + h_c0 <=> gly_s0 + h_s0
53.53%,-12.49,GLYtx_M,gly_c0 + h_c0 <=> gly_x0 + h_x0
0.42%,-0.09743,Straw_Biomass_M,0.0321 ala_DASH_L_c0 + 0.0123 arg_DASH_L_c0 + 0.0417 asp_DASH_L_c0 + 33.259 atp_c0 + 2.0789 cellulose_c0 + 0.1721 conoh_c0 + 0.2065 coumoh_c0 + 0.0071 ctp_c0 + 0.0033 cys_DASH_L_c0 + 0.0009 datp_c0 + 0.0007 dctp_c0 + 0.0007 dgtp_c0 + 0.0009 dttp_c0 + 0.00285 fa_os_r0 + 0.0643 fru_DASH_B_c0 + 0.0546 glc_DASH_A_c0 + 0.0385 glu_DASH_L_c0 + 0.0282 gly_c0 + 0.0071 gtp_c0 + 0.057 his_DASH_L_c0 + 0.014 ile_DASH_L_c0 + 0.0234 leu_DASH_L_c0 + 0.0153 lys_DASH_L_c0 + 0.0076 met_DASH_L_c0 + 0.000191 pa_os_r0 + 0.0034 pc_os_r0 + 0.00178 pe_os_r0 + 0.0133 phe_DASH_L_c0 + 0.0225 pro_DASH_L_c0 + 0.00041 ptd1ino_os_r0 + 0.0219 ser_DASH_L_c0 + 0.1475 sinoh_c0 + 0.3249 starch_s0 + 0.1049 sucr_c0 + 0.0192 thr_DASH_L_c0 + 0.0298 triglyc_os_r0 + 0.0016 trp_DASH_L_c0 + 0.0077 tyr_DASH_L_c0 + 0.1256 udparab_c0 + 0.4018 udpg_c0 + 0.0229 udpgal_c0 + 0.5033 udpglcur_c0 + 0.572 udpxyl_c0 + 0.009 utp_c0 + 0.0225 val_DASH_L_c0 --> 33.259 adp_c0 + 1.626 h_c0 + 33.259 pi_c0 + 1.626 udp_c0
